In [3]:
search_for = 125
start_from = 0
threads = 10
thread = 9
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.002378225326538086
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 35
all cases: 320327931
type: [1, 1, 1, 1, 125] 125
cases of this type: 244140625
100000 13.438771626115372
200000 25.706739915725343
300000 24.145387360404683
400000 24.18351880595378
500000 26.461619028247437
600000 25.340035293798085
700000 23.758738788926323
800000 24.291600501682304
900000 24.253838924079357
1000000 11.75500171348661
1100000 22.77700388226689
1200000 24.495265578402435
1300000 24.855907714838903
1400000 25.407708730744414
1500000 25.523209320735972
1600000 24.809258782448396
1700000 24.727777214749047
1800000 24.431890753593322
1900000 24.211437725937277
2000000 8.978349283049441
2100000 25.150905955045214
2200000 28.52181189527857
2300000 28.178978534272584
2400000 28.089935995033674
2500000 27.889767962501658
2600000 26.095462846819828
2700000 27.925971643072543
2800000 26.44493560790914
2900000 25.839635634764754
3000

29400000 24.25598223082861
29500000 24.103722504316753
29600000 9.890481237091368
29700000 22.446832257454147
29800000 23.406521558435717
29900000 23.80612445847355
30000000 21.827849480150004
30100000 21.95008082111612
30200000 20.781433412410625
30300000 9.869999988595293
30400000 20.50457071290692
30500000 21.0687035162416
30600000 23.340323842795403
30700000 22.93515303310808
30800000 23.78071097027499
30900000 23.31980407081071
31000000 19.072729048594955
31100000 16.462641764879336
31200000 23.062172596653976
31300000 22.532610161425154
31400000 23.1184613501462
31500000 18.119069524360942
31600000 15.111626252187653
31700000 23.06790765472415
31800000 22.010051488103425
31900000 22.592510049144792
32000000 20.918902523901487
32100000 20.02671819569885
32200000 19.047066356770905
32300000 10.628127964498518
32400000 20.35675616022922
32500000 21.600445150806614
32600000 20.808447859655868
32700000 20.885181848304704
32800000 22.70122374910721
32900000 23.379444167331044
33000000 

59200000 17.032124255082916
59300000 18.35528295050251
59400000 19.065733376820646
59500000 18.4240045162602
59600000 8.455689237254576
59700000 17.706874534097786
59800000 18.649439596826156
59900000 18.030066797592458
60000000 17.56875627625441
60100000 9.629804725221836
60200000 19.140699016360376
60300000 20.30638900714161
60400000 20.17499583605631
60500000 20.203618082262544
60600000 21.044337201022714
60700000 20.264366699729756
60800000 20.582166219064142
60900000 20.079923626178495
61000000 19.4399748105715
61100000 8.755502799622652
61200000 17.654449719722628
61300000 18.515748297677103
61400000 18.334456006764036
61500000 15.998263283085763
61600000 10.28559421899019
61700000 17.868837757311702
61800000 18.41678516674374
61900000 18.792654203772265
62000000 14.639480673032038
62100000 13.681003972821136
62200000 20.053273445475515
62300000 20.45993895650597
62400000 19.59175938075705
62500000 21.08298765061183
62600000 19.34238555656605
62700000 18.553952452358747
62800000 

89000000 16.400257245558215
89100000 15.278259318612108
89200000 9.045004170842539
89300000 17.149509183158692
89400000 15.795569783347066
89500000 15.879252785274275
89600000 16.995279618960414
89700000 17.21257250933828
89800000 18.694848239262203
89900000 17.456144089863294
90000000 18.338789276675413
90100000 17.142655815962172
90200000 16.5844405732448
90300000 16.21565023609818
90400000 16.43177191599442
90500000 15.372306420454013
90600000 8.27506954556018
90700000 16.218791252394087
90800000 13.362919837150583
90900000 8.322104671492182
91000000 15.847558583038655
91100000 8.899285889938632
91200000 15.860323208492165
91300000 16.005800548260833
91400000 15.757636807459221
91500000 15.893484701562084
91600000 16.990049864203765
91700000 17.23191363092226
91800000 17.75114802408408
91900000 17.030254764038414
92000000 18.50827862888034
92100000 16.751454030553106
92200000 15.438148090800263
92300000 16.566807911818245
92400000 15.696558841706587
92500000 16.2022264746249
9260000

118000000 1.5811938696741086
118100000 1.5403460584506425
118200000 1.7449206957548737
118300000 1.6360083284515652
118400000 1.524281291732202
118500000 1.4754146517737132
118600000 1.5279582827082794
118700000 1.527933275514353
118800000 1.5726609198948165
118900000 1.660777675594188
119000000 1.5893560372205053
119100000 1.5183013181539595
119200000 1.6388078262748116
119300000 1.8097226841298308
119400000 1.5784688275136252
119500000 1.5239587705540003
119600000 1.7286176843407215
119700000 1.6826316514000654
119800000 1.7509733045408415
119900000 1.5892242438004704
120000000 1.6432024205019773
120100000 1.6358938990087681
120200000 1.6010549234784464
120300000 1.4899207079888543
120400000 1.4911002555299653
120500000 1.4798104473450218
120600000 1.6955176817857405
120700000 1.582037958882178
120800000 1.6251177089666895
120900000 1.564298240688479
121000000 1.5798179665503043
121100000 1.5618649617278855
121200000 1.5550875789830982
121300000 1.7293499115714643
121400000 1.5544561

...

204400000 8.272415309015878
204500000 9.410402340057814
204600000 8.441905454781663
204700000 9.755840495246614
204800000 4.175453583942676
204900000 9.9841537927639
205000000 8.70477123496111
205100000 10.20227762112442
205200000 9.46045860987473
205300000 8.97861298399249
205400000 5.118412124049689
205500000 7.729570724530894
205600000 9.934708453988618
205700000 7.905394582833943
205800000 8.69346375935983
205900000 7.617797459048291
206000000 7.796127745533701
206100000 3.3358007279019
206200000 7.825174835317674
206300000 7.773485437486913
206400000 8.267035286685115
206500000 8.096032932434378
206600000 8.869487776318298
206700000 7.962791745695714
206800000 4.5252719243692425
206900000 8.821845412225874
207000000 8.260057264754035
207100000 9.172880609077632
207200000 8.44656098996226
207300000 8.125435716167301
207400000 5.314324439738734
207500000 8.913372941589776
207600000 8.211747139503368
207700000 7.769015875353313
207800000 8.462700265669133
207900000 7.3359284573744485

233700000 6.437817592353131
233800000 6.82165905639059
233900000 6.4933236201416396
234000000 7.2958423221862505
234100000 6.558032189895626
234200000 6.897409792920827
234300000 4.416415642988265
234400000 5.498008243152635
234500000 3.5272584790186823
234600000 6.753597867874054
234700000 6.318292547604459
234800000 6.813390029425212
234900000 6.404001381696834
235000000 6.625211802357032
235100000 5.9886589125938645
235200000 6.7657051503245835
235300000 6.166736788818421
235400000 2.216504517815176
235500000 6.841113573434644
235600000 6.276406170027173
235700000 6.538202020897582
235800000 6.302108455281912
235900000 7.087223619313273
236000000 6.33489585460875
236100000 6.702775330681677
236200000 6.462982394740288
236300000 2.7792841570194042
236400000 4.461196871522024
236500000 5.656114684256313
236600000 6.343098265795839
236700000 6.741168242711546
236800000 6.419424339718857
236900000 6.739227445389524
237000000 5.966185110693675
237100000 6.498862251006677
237200000 6.1764

262300000 1.6285217159955463
262400000 1.5731733774468282
262500000 1.980250836767765
262600000 1.1158601588356363
262700000 0.5267597874231266
262800000 0.5041010693569462
262900000 1.1340456471174294
263000000 2.126945240482261
263100000 1.568542994705522
263200000 1.7071763934622686
263300000 1.724186984091839
263400000 1.6573874170624814
263500000 1.7108356528127364
263600000 1.646767120178343
263700000 1.9508436911275546
263800000 1.7957242119050436
263900000 1.7494919300720044
264000000 1.654498497810602
264100000 1.5724055595106663
264200000 1.5572797343453728
264300000 1.6143496139835891
264400000 1.7555311241310663
264500000 1.4931241605760548
264600000 0.7613932937438469
264700000 0.4831969248322288
264800000 0.6568976348012169
264900000 1.4376492738658904
265000000 1.923605370997996
265100000 1.6011317032505852
265200000 1.4999644355898638
265300000 1.6244461170655489
265400000 1.6056017312193414
265500000 1.6317866226629036
265600000 1.6481449662035337
265700000 1.897239582

290700000 0.9221050258245984
290800000 0.9559464259793116
290900000 0.9051337603864432
291000000 0.9809206075772763
291100000 0.9996956279936546
291200000 1.0042325879847422
291300000 0.8674199939093669
291400000 0.8910438656558077
291500000 0.9278151083363175
291600000 0.8294405629071773
291700000 0.77379143001835
291800000 0.9760934732151337
291900000 0.5386071388509512
292000000 0.245015428607036
292100000 0.2428086222259899
292200000 0.5350818005009174
292300000 0.9935997942476384
292400000 0.7448637111567962
292500000 0.8881206888572553
292600000 0.9064114067753282
292700000 0.7951968481693712
292800000 0.8905757427505732
292900000 0.8836924980780947
best so far: 0
type: [1, 1, 1, 25, 5] 125
cases of this type: 9765625
293000000 0.9558249431982769
293100000 0.8465969236646393
293200000 0.7641369404821993
293300000 0.7962544910963978
293400000 0.8081459673118828
293500000 0.6880169417222461
293600000 0.6064565633302279
293700000 0.7414245095709086
293800000 0.7347079547406382
29390

best so far: 0
type: [1, 1, 25, 5, 1] 125
cases of this type: 78125
317200000 0.019378226967276466
best so far: 0
type: [1, 1, 125, 1, 1] 125
cases of this type: 15625
best so far: 0
type: [1, 5, 1, 1, 25] 125
cases of this type: 1953125
317300000 0.02916981005608771
317400000 0.06180444335108798
317500000 0.051344698032312924
317600000 0.04617736428770886
317700000 0.04807073664641579
317800000 0.041858441093362696
317900000 0.040230499547796124
318000000 0.043917619034796955
318100000 0.032542118493054314
318200000 0.014089453591522243
318300000 0.01421061944068869
318400000 0.013520142891422244
318500000 0.024874736875693006
318600000 0.03542379335276882
318700000 0.030442599149354963
318800000 0.029905466518787543
318900000 0.025657109092003107
319000000 0.02500458195218775
319100000 0.023349559629734357
319200000 0.0224930395970126
best so far: 0
type: [1, 5, 1, 5, 5] 125
cases of this type: 390625
319300000 0.014896086236499417
319400000 0.00840949710474809
319500000 0.0066506047